In [183]:
import pandas as pd

# Cargar los archivos
product_df = pd.read_excel('DIM_PRODUCT (1).xlsx')
calendar_df = pd.read_excel('DIM_CALENDAR (2).xlsx')
category_df = pd.read_csv('DIM_CATEGORY (2).csv')
segment_df = pd.read_excel('DIM_SEGMENT (1).xlsx')
sales_df = pd.read_csv('FACT_SALES (1).csv')

# Eliminar espacios, convertir a mayúsculas y limpiar saltos de línea
category_df['CATEGORY'] = category_df['CATEGORY'].str.strip().str.upper().str.replace('\n', '', regex=True)
segment_df['FORMAT'] = segment_df['FORMAT'].str.strip().str.upper()
segment_df['ATTR3'] = segment_df['ATTR3'].str.strip().str.upper()
product_df['FORMAT'] = product_df['FORMAT'].str.strip().str.upper()
product_df['ATTR3'] = product_df['ATTR3'].str.strip().str.upper()

# Rellenar valores nulos
product_df['ATTR1'] = product_df['ATTR1'].fillna('NO DEFINIDO')
product_df['ATTR3'] = product_df['ATTR3'].fillna('NO DEFINIDO')


In [140]:

# Eliminar duplicados
product_df.drop_duplicates(inplace=True)
calendar_df.drop_duplicates(inplace=True)
category_df.drop_duplicates(inplace=True)
segment_df.drop_duplicates(inplace=True)
sales_df.drop_duplicates(inplace=True)



In [187]:
# --- Uniones entre tablas ---
# Unir product con category
product_df['CATEGORY'] = product_df['CATEGORY'].astype(int)
category_df['ID_CATEGORY'] = category_df['ID_CATEGORY'].astype(int)
product_cat_df = pd.merge(product_df, category_df, left_on='CATEGORY', right_on='ID_CATEGORY', how='left')



In [181]:
# Renombrar para la unión con segment
product_cat_df.rename(columns={'ID_CATEGORY': 'CATEGORY_ID'}, inplace=True)
segment_df.rename(columns={'CATEGORY': 'CATEGORY_ID'}, inplace=True)


In [170]:
# Unión con segment
product_cat_seg_df = pd.merge(product_cat_df, segment_df,
                              on=['CATEGORY_ID', 'ATTR1', 'ATTR2', 'ATTR3', 'FORMAT'],
                              how='left')

In [172]:
# Preparar claves para la unión con ventas
sales_df['ITEM_CODE'] = sales_df['ITEM_CODE'].astype(str)
product_cat_seg_df['ITEM'] = product_cat_seg_df['ITEM'].astype(str)


In [174]:
# Unión con ventas
sales_merged_df = pd.merge(sales_df, product_cat_seg_df, left_on='ITEM_CODE', right_on='ITEM', how='left')


In [177]:
# Unión con calendario
final_df = pd.merge(sales_merged_df, calendar_df, on='WEEK', how='left')

In [179]:
# --- Transformaciones finales ---
final_df['DATE'] = pd.to_datetime(final_df['DATE'], errors='coerce')
final_df['YEAR_MONTH'] = final_df['DATE'].dt.to_period('M').astype(str)


In [185]:
# Guardar archivo final
final_df.to_excel('Base_Final.xlsx', index=False)